In [ ]:
import pandas as pd
import warnings
import numpy as np
import re # regular expressions
from tqdm import tnrange, tqdm_notebook, tqdm
from textblob import TextBlob
warnings.filterwarnings("ignore")

In [ ]:
bitcoin_tweets_df=pd.read_csv('Data/tweets/Bitcoin_tweets.csv')
dogecoin_tweets_df=pd.read_csv('Data/tweets/Dogecoin_tweets.csv')

In [ ]:
bitcoin_tweets_df_min=bitcoin_tweets_df[['date','text','is_retweet']]
dogecoin_tweets_df_min=dogecoin_tweets_df[['created_at','tweet','language','retweet']]

In [ ]:
bitcoin_tweets_df_min['date']=pd.to_datetime(bitcoin_tweets_df_min['date'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

dogecoin_tweets_df_min['created_at'] = dogecoin_tweets_df_min['created_at'].str.split('+').str[0]
dogecoin_tweets_df_min['created_at']=pd.to_datetime(dogecoin_tweets_df_min['created_at'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

In [ ]:
bitcoin_tweets_df_min = bitcoin_tweets_df_min.dropna(subset=['date'])
dogecoin_tweets_df_min = dogecoin_tweets_df_min.dropna(subset=['created_at'])

In [ ]:
dogecoin_tweets_df_min_eng=dogecoin_tweets_df_min.loc[dogecoin_tweets_df_min['language']=='en']
dogecoin_tweets_df_min_eng=dogecoin_tweets_df_min_eng.drop(['language'], axis=1)
dogecoin_tweets_df_min_oth=dogecoin_tweets_df_min.loc[dogecoin_tweets_df_min['language']!='en']

In [ ]:
dogecoin_tweets_df_min_eng.columns=['date', 'text', 'is_retweet']

In [14]:
bitcoin_tweets_df_min['text']=bitcoin_tweets_df_min['text'].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))
dogecoin_tweets_df_min_eng['text']=dogecoin_tweets_df_min_eng['text'].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))

In [15]:
bitcoin_tweets_df_min['text'] = bitcoin_tweets_df_min['text'].replace(r"http\S+", ' ', regex=True).replace(r'[^a-zA-Z]+', ' ', regex=True).replace(r'\s+', ' ',regex=True).replace(r'^b\s+','',regex=True)

In [16]:
dogecoin_tweets_df_min_eng['text'] = dogecoin_tweets_df_min_eng['text'].replace(r"http\S+", ' ', regex=True).replace(r'[^a-zA-Z]+', ' ', regex=True).replace(r'\s+', ' ',regex=True).replace(r'^b\s+','',regex=True)

In [ ]:
def deEmojify(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)


    #data = data.fillna('')
    #stemmer = PorterStemmer()
    #lemmatizer = WordNetLemmatizer()
    #stopwordslist = stopwords.words('english')
    #stopwordslist.remove('not')
    #stopwordslist = set(stopwordslist)
    #for i in range(data[''].count()):
    #    data['Mail'][i] = ' '.join([
    #        stemmer.stem(word) for word in str(data['Mail'][i]).lower().split()
    #        if word not in stopwordslist
    #    ])
    #    data['Subject'][i] = ' '.join([
    #        stemmer.stem(word) for word in str(data['Subject'][i]).lower().split()
    #        if word not in stopwordslist
    #    ])
    #data

def cleaning(d):
    for i,s in enumerate(tqdm(d['text'])):
        text = d.loc[i, 'text']
        text = text.replace("#", "")
        text = re.sub('https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', '', text, flags=re.MULTILINE)
        text = re.sub('@\\w+ *', '', text, flags=re.MULTILINE)
        text = deEmojify(text)
        d.loc[i, 'text'] = text
    return d

In [17]:
bitcoin_tweets_df_min.to_csv('Data/tweets/Bitcoin_tweets_min.csv', header=True, encoding='utf-8',index=False)
dogecoin_tweets_df_min_eng.to_csv('Data/tweets/Dogecoin_tweets_min_eng.csv', header=True, encoding='utf-8',index=False)
dogecoin_tweets_df_min_oth.to_csv('Data/tweets/Dogecoin_tweets_min_oth.csv', header=True, encoding='utf-8',index=False)